In [5]:
!pip install torch torchvision torchaudio

   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB 991.0 kB/s eta 0:02:42
   ---------------------------------------- 0.1/159.8 MB 919.0 kB/s eta 0:02:54
   ---------------------------------------- 0.2/159.8 MB 1.5 MB/s eta 0:01:47
   ---------------------------------------- 0.3/159.8 MB 1.7 MB/s eta 0:01:34
   ---------------------------------------- 0.4/159.8 MB 2.2 MB/s eta 0:01:13
   ---------------------------------------- 0.7/159.8 MB 2.8 MB/s eta 0:00:57
   ---------------------------------------- 0.9/159.8 MB 3.4 MB/s eta 0:00:47
   ---------------------------------------- 1.2/159.8 MB 3.7 MB/s eta 0:00:44
   ---------------------------------------- 1.5/159.8 MB 4.2 MB/s eta 0:00:39
   ---------------------------------------- 2.0/159.8 MB 4.8 MB/s eta 0:00:33
    --------------------------------------- 2.3/159.8 MB 5.2 MB/s eta 0:00:31
    --------------------------------------- 2.6/159.8 MB 5.4 MB/s e

In [7]:
import torch
import torch.nn as nn
import torch.functional as F
import numpy as np

In [9]:
import torchvision.datasets as datasets

In [13]:
dataset_path="~/datasets"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dim = 784 # each image is 28*28 = 784 pixels
batch_size = 100
hidden_dim = 400
latent_dim = 200
learning_rate = 1e-3
epochs = 30

In [14]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# MNIST
mnist_transform = transforms.Compose([
    transforms.ToTensor()
])

kwargs = {'num_workers':1, 'pin_memory':True}
train_dataset = MNIST(dataset_path, transform=mnist_transform,train=True, download=True)
test_dataset = MNIST(dataset_path, transform=mnist_transform, train=False, download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True,**kwargs)
test_loader = DataLoader(dataset=test_dataset, batch_size = batch_size,shuffle=True , **kwargs)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:43<00:00, 229296.50it/s]


Extracting C:\Users\Madan/datasets\MNIST\raw\train-images-idx3-ubyte.gz to C:\Users\Madan/datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 85058.30it/s]


Extracting C:\Users\Madan/datasets\MNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\Madan/datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:06<00:00, 253985.73it/s]


Extracting C:\Users\Madan/datasets\MNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\Madan/datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 2027514.77it/s]

Extracting C:\Users\Madan/datasets\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\Madan/datasets\MNIST\raw



In [17]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim , latent_dim):
        super(Encoder, self).__init__()
        # Hidden layers used to process the inputs before converting to latents
        self.hidden_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(0.2),
            nn.Linear(hidden_dim , hidden_dim),
            nn.LeakyReLU(0.2)
        )
        # Laten representations encoded into mean and lo variance vector
        self.mean = nn.Linear(hidden_dim, latent_dim)
        self.log_variance = nn.Linear(hidden_dim, latent_dim)
        self.training = True
    def forward(self, x):
        hidden = self.hidden_layer(x)
        mean = self.mean(hidden)
        log_variance = self.log_variance(hidden)
        return mean, log_variance

In [22]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder , self).__init__()
        self.hidden_layer = nn.Sequential(
            nn.Linear(latent_dim , hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim , hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim , output_dim)
        )
    def forward(self, x):
        hidden = self.hidden_layer(x)
        x_hat = torch.sigmoid(hidden)
        return x_hat

In [23]:
class Model(nn.Module):
    def __init__(self, Encoder , Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    def reparameterization(self , mean , variance):
        z = mean + variance * epsilon
        return z
    def forward(self, x):
        mean , log_variance = self.Encoder(x)
        # Use the reparameterization trick to keep randomness differentiable 
        z = self.reparameterization(mean, torch.exp(0.5 * log_variance))
        x_hat = self.Decoder(z)
        return x_hat, mean , log_variance
        

In [24]:
encoder = Encoder(input_dim=data_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim=hidden_dim, output_dim = data_dim)

model = Model(Encoder=encoder, Decoder=decoder).to(device)

In [ ]:
from torch.optim import Adam
def bce_loss(x , x_hat, mean , log_variance):
    rec